## Model description

This benchmark models a water tank which leaks, i.e., it has a constant outflow. The tank can be refilled from an unlimited external resource with a constant inflow that is larger than the outflow. The PLC controller triggers refilling (by switching a pump on) if a sensor indicates a low water level ($h\leq 6$). If the water level is high ($h \geq 12$) the controller stops refilling (switches the pump off).

Adding the controller introduces two controller input variables for low and high water levels, variables for the actuator (pump) state in the plant and the controller, and a variable to store the controller mode.

Furthermore, a new clock is added to model the PLC cycle time. Besides the controller we also model a user which can manually switch
the pump on and off as far as the water level allows it. In our implementation, the user constantly toggles between the pump states on and off. We analyse the system behaviour over a global time horizon of $40$ seconds using a PLC cycle time of $2$ seconds.

The dynamics of the waterlevel of the tank is defined by the following differential equations:

$$
    \dot{h} = \left\{ \begin{array}{l l} \phantom{-}2 & \quad \text{if the pump is on and the tank is not full}\\ \phantom{-}2 & \quad \text{if the tank is full}\\ -1.5 & \quad \text{if the pump is off}\\ \end{array} \right.
$$

This model has 12 continuous variables, 11 modes and 34 discrete jumps.

The model parameters and description are taken from the [HyPro collection of continuous and hybrid system benchmarks](https://ths.rwth-aachen.de/research/projects/hypro/benchmarks-of-continuous-and-hybrid-systems/), see the [Leaking Tank with Controller model](https://ths.rwth-aachen.de/research/projects/hypro/leaking-tank-with-controller/).

---

*References:*

[1] J. Nellen. Analysis and synthesis of hybrid systems in engineering applications. PhD Thesis, RWTH Aachen University, 2016.

[2] S. Schupp et al.. Divide and conquer: Variable set separation in hybrid systems reachability analysis. Submitted to (QAPL’17).

In [1]:
using Revise # to debug
using Reachability, HybridSystems, MathematicalSystems, LazySets, LinearAlgebra, SX
using Plots, LaTeXStrings

┌ Warning: Package LazySets does not have Expokit in its dependencies:
│ - If you have LazySets checked out for development and have
│   added Expokit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LazySets
│ Loading Expokit into LazySets from project dependency, future warnings for LazySets are suppressed.
└ @ nothing nothing:840


In [11]:
file = "leaking_tank_spaceEx_separateControllerAndPlant_with_timer_flat.xml"
H = readsxmodel(file, raw_dict=true)

BoundsError: BoundsError: attempt to access 11-element Array{Array{Expr,1},1} at index [14]

In [5]:
H

In [ ]:
function leaking_tank(;X0=Singleton([510.0, 20.0, 20.0]),
                      T=5.0,
                      ε=1e-6)

    # automaton structure
    automaton = LightAutomaton(4)

    # rod_1  : x' = Ax + b
    A = [0.1 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
    b = [-56.0, 1.0, 1.0]
    X = HPolyhedron([HalfSpace([-1.0, 0.0, 0.0], -510.0)]) # x >= 510
    rod_1 = ConstrainedAffineContinuousSystem(A, b, X)
    
    # no_rods
    b = [-50.0, 1.0, 1.0]
    X = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 550.0)])   # x <= 550
    no_rods = ConstrainedAffineContinuousSystem(A, b, X)

    # rod_2
    b = [-60.0, 1.0, 1.0]
    X = HPolyhedron([HalfSpace([-1.0, 0.0, 0.0], -510.0)])   # x >= 510
    rod_2 = ConstrainedAffineContinuousSystem(A, b, X)

    # shut_down
    X = Universe(2)
    shut_down = ConstrainedContinuousIdentitySystem(3, X)

    # modes
    modes = [rod_1, no_rods, rod_2, shut_down]

    add_transition!(automaton, 1, 2, 1)
    add_transition!(automaton, 2, 1, 2)
    add_transition!(automaton, 2, 3, 3)
    add_transition!(automaton, 3, 2, 4)
    add_transition!(automaton, 2, 4, 5)

    # guards
    G12 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 510.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0], -510.0 + ε)]) # x = 510

    G21 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 550.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0], -550.0 + ε),  # x = 550
                       HalfSpace([0.0, -1.0, 0.0], -20.0)])      # c1 >= 20

    G23 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 550.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0], -550.0 + ε),  # x = 550
                       HalfSpace([0.0, 0.0, -1.0], -20.0)])      # c2 >= 20

    G32 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 510.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0], -510.0 + ε)]) # x = 510

    G24 = HPolyhedron([HalfSpace([1.0, 0.0, 0.0], 550.0 + ε),
                       HalfSpace([-1.0, 0.0, 0.0], -550.0 + ε),  # x = 550
                       HalfSpace([1.0, 0.0, 0.0], 20.0),         # c1 < 20
                       HalfSpace([0.0, 1.0, 0.0], 20.0)])        # c2 < 20
    
    resetmaps = [ConstrainedResetMap(2, G12, Dict(2=>0.0)), 
                 ConstrainedIdentityMap(2, G21),
                 ConstrainedIdentityMap(2, G23),
                 ConstrainedResetMap(2, G32, Dict(3=>0.0)),
                 ConstrainedIdentityMap(2, G24)]

    # switching
    switchings = [AutonomousSwitching()]

    ℋ = HybridSystem(automaton, modes, resetmaps, switchings)

    # initial condition in "off_off" mode
    initial_condition = [(1, X0)]

    problem = InitialValueProblem(ℋ, initial_condition)

    options = Options(:mode=>"reach", :T=>T, :plot_vars=>[1, 3],
                      :project_reachset=>false)

    return (problem, options)
end

## Reachability settings

We consider an initial set of

$$
    h = 10 \\
    \text{low} = 1 \\
    \text{high} = 0 \\
    P = P\_{\text{plc}} = 1 \\
    P\_{\text{on}} = P\_{\text{on}}\_{\text{plc}} = 0\\
    P\_{\text{off}} = P\_{\text{off}}\_{\text{plc}} = 1\\
    \text{nextSfcLoc} = 1\\
    \text{cycle_time} = 0\\
    \text{global_time} = 0    
$$
and the starting location `clock_switch_on_in_comm`, a time horizon $\text{global\_time}=40s$, a PLC cycle duration of $\delta = 2s$, and a time step $r=0.01s$ .

In [ ]:
# settings
h = 10
low = 1
high = 0
P = P_plc = 1
P_on = P_on_plc = 0
P_off = P_off_plc = 1
nextSfcLoc = 1
cycle_time = 0
global_time = 0

LeakingTank, options = leaking_tank()

## Results

In [ ]:
using Polyhedra

In [ ]:
# BROKEN.. why?

#=
@time begin
    opC = BFFPSV18(:δ=>0.1)
    opD = LazyDiscretePost()
    sol = solve(RodReactor, options, opC, opD)
end;

the exact support vector of an intersection is not implemented
=#

In [ ]:
# similar to approximating discrete post ... gives big overapprox error
# with default options
using LazySets.Approximations

@time begin
    opC = BFFPSV18(:δ=>0.1)
    opD = ApproximatingDiscretePost()
    sol = solve(RodReactor, options, opC, opD)
end;

In [ ]:
# similar to approximating discrete post ... gives big overapprox error
# with default options. we try to use oct direction but it doesn't use them
using LazySets.Approximations

@time begin
    opC = BFFPSV18(:δ=>0.1)
    opD = LazyDiscretePost(:check_invariant_intersection=>true,
                           :overapproximation=>OctDirections,
                           :lazy_R⋂I=>false,
                           :lazy_R⋂G=>false,
                           :lazy_A⌜R⋂G⌟⋂I=>false)
    sol = solve(RodReactor, options, opC, opD)
end;

In [ ]:
# using Concrete Discrete Post
@time begin
    opC = BFFPSV18(:δ=>0.01)
    opD = ConcreteDiscretePost()
    sol = solve(RodReactor, options, opC, opD) # does it produce 
end;

In [ ]:
using LazySets.Approximations: project, overapproximate

In [ ]:
Xkproj = [project(sol.Xk[i].X, [1, 3], Hyperrectangle) for i in eachindex(sol.Xk)];
plot(Xkproj, xlab="x", ylab="c2")

In [ ]:
Xkproj = [project(sol.Xk[i].X, [1, 2], Hyperrectangle) for i in eachindex(sol.Xk)];
plot(Xkproj, xlab="x", ylab="c1")

In [ ]:
Xkproj = [project(sol.Xk[i].X, [2, 3], Hyperrectangle) for i in eachindex(sol.Xk)];
plot(Xkproj, xlab="c1", ylab="c2")